<a href="https://colab.research.google.com/github/TurkuNLP/Deep_Learning_in_LangTech_course/blob/master/hf_trainer_mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!wget -nc https://raw.githubusercontent.com/TurkuNLP/sentiment-target-corpus/main/sentiment-target-fi.tsv
!pip3 install transformers

File ‘sentiment-target-fi.tsv’ already there; not retrieving.



In [6]:
import re
import json
import random

label_names=["positive","negative","reject","neither"]
data=[]
with open("sentiment-target-fi.tsv") as f:
    for line in f:
        line=line.rstrip("\n")
        if not line or line.startswith("#"):
            continue
        cols=line.split("\t")
        if len(cols)!=5:
            continue
        data.append(cols)
random.shuffle(data)
with open("sentiment-data.jsonl","wt") as f:
    for cols in data:
        txt=cols[1]
        item={"label":label_names.index(cols[2]),"text":cols[1]}
        print(json.dumps(item,ensure_ascii=False,sort_keys=True),file=f)


In [7]:
!pip3 install datasets
!pip3 install transformers

In [8]:
import datasets

fname="sentiment-data.jsonl"
dset=datasets.load_dataset('json',
                           data_files={"everything":fname},
                           split={"train":"everything[:80%]","validation":"everything[80%:90%]","test":"everything[90%:]"},
                           features=datasets.Features({
                               "label":datasets.ClassLabel(names=["positive","negative","neither","reject"]),
                               "text":datasets.Value("string")})
                           )                           


Using custom data configuration default-b4da998532212d40


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-b4da998532212d40/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
import transformers
tokenizer=transformers.AutoTokenizer.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1")

dset=dset.map(lambda x: tokenizer(x["text"],truncation=True))

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [10]:
dset["train"][0]

{'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'input_ids': [102,
  33402,
  4096,
  5617,
  743,
  111,
  8824,
  629,
  5571,
  16307,
  50051,
  50073,
  12355,
  2377,
  320,
  23548,
  5571,
  499,
  16307,
  50051,
  50073,
  12355,
  2377,
  251,
  4001,
  7527,
  462,
  111,
  103],
 'label': 0,
 'text': 'Mieluumin toivon esim. Mestaruuden <TARGET>Ipalle</TARGET> kun vaikkapa HIFKille.',
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [11]:
dset["train"][0]

{'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'input_ids': [102,
  33402,
  4096,
  5617,
  743,
  111,
  8824,
  629,
  5571,
  16307,
  50051,
  50073,
  12355,
  2377,
  320,
  23548,
  5571,
  499,
  16307,
  50051,
  50073,
  12355,
  2377,
  251,
  4001,
  7527,
  462,
  111,
  103],
 'label': 0,
 'text': 'Mieluumin toivon esim. Mestaruuden <TARGET>Ipalle</TARGET> kun vaikkapa HIFKille.',
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [21]:
import torch

class MLPConfig(transformers.PretrainedConfig):

    pass

class MLP(transformers.PreTrainedModel):

    def __init__(self,config):
        super().__init__(config)
        self.vocab_size=config.vocab_size
        self.hidden=torch.nn.Linear(in_features=self.vocab_size,out_features=config.hidden_size)
        self.output=torch.nn.Linear(in_features=config.hidden_size,out_features=config.nlabels)
        

    def forward(self,input_ids,labels=None,**kwargs):
        #print(input_ids.shape)
        #print(labels)
        #print(kwargs.keys())
        batch_size=input_ids.shape[0]
        input=torch.zeros((batch_size,self.vocab_size),dtype=torch.float,device=input_ids.device)
        ones=torch.ones_like(input_ids,dtype=torch.float)
        input.scatter(1,input_ids,ones)
        projected=torch.tanh(self.hidden(input))
        logits=self.output(projected)
        if labels is not None:
            loss=torch.nn.CrossEntropyLoss()
            return (loss(logits,labels),logits)
        else:
            return logits

mlp_config=MLPConfig(vocab_size=100000,hidden_size=10,nlabels=4)
mlp=MLP(mlp_config)

trainer_args=transformers.TrainingArguments("xxx",remove_unused_columns=True)
data_collator=transformers.DataCollatorWithPadding(tokenizer)
trainer=transformers.Trainer(model=mlp,args=trainer_args,train_dataset=dset["train"],eval_dataset=dset["validation"],data_collator=data_collator)
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set  don't have a corresponding argument in `MLP.forward` and have been ignored: token_type_ids, text, attention_mask.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1864
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization step

Step,Training Loss
500,1.349600


Saving model checkpoint to xxx/checkpoint-500
Configuration saved in xxx/checkpoint-500/config.json
Model weights saved in xxx/checkpoint-500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=699, training_loss=1.3476547219381483, metrics={'train_runtime': 15.576, 'train_samples_per_second': 359.015, 'train_steps_per_second': 44.877, 'total_flos': 1615431228576.0, 'train_loss': 1.3476547219381483, 'epoch': 3.0})